In [1]:
# Import modules
import sys

In [2]:
# Import custom modules
sys.path.append('../')

from src.models.upload_data import *

In [3]:
# Set notebook attributes
pd.set_option('display.max_colwidth', 500)
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', 300)
pd.set_option('display.float_format', lambda x: '%.2f' % x)

In [4]:
# Set magic attributes
%matplotlib inline
%load_ext autoreload
%autoreload 2

In [5]:
# Configuration
args = argparse.ArgumentParser()
args.add_argument("--config", default="../params.yaml")
parsed_args = vars(args.parse_args(""))
config_path = parsed_args['config']

config = read_params(config_path)
project_dir = config["project_dir"]
ftd_clean_data_path = config["processed_data_config"]["ftd_clean_data_parquet"]
scored_limits_risk_review_data_path_parquet = config["processed_data_config"]["scored_limits_risk_review_data_parquet"]

### First Time Defaulters Monitoring

#### Latest Limits

In [6]:
%%time
# Data loading
df_scoring_stabilisation = pd.read_parquet(project_dir + scored_limits_risk_review_data_path_parquet)
df_scoring_stabilisation.sample(2)

CPU times: total: 31.2 ms
Wall time: 113 ms


,mobile_number,store_number,most_recent_trx_date_past_30_days,last_trx_date,actual_trx_days,approx_30_days_trx_val,expected_trx_days,page_active_days,inference_col,days_since_last_trx,transacted_last_5_days,weight_till_recency,national_id,is_iprs_validated,idm_recommendation,idm_limit,client_mobile_number,loan_count,loan_status,term_frequency,principal_disbursed,principal_repaid,disbursed_on_date,expected_matured_on_date,closed_on_date,due_date_fixed,end_rollover_date_fixed,expected_dpd90,days_past_due,bloom_version,loan_repayment_status,src_crdt_score,max_principal_amount,max_loan_disbursement_date,count_good_loans,good_loans_repayment_ratio,num_days_since_last_disbursement,weight_dpd,max_days_past_due,loan_count_past_3_months,count_7_day_loans,avg_loan_tenure,count_7_day_loans_paid_upto_rollover,good_loans_repayment_ratio(7_day_loans),minimum_7_day_principal_disbursed,total_sum_7_day_principal_disbursed,total_sum_1_day_principal_disbursed,avg_7_day_principal_disbursed,days_past_end_rollover,any_bloom2_1day,total_outstanding_sum,safaricom_loan_balance_sum,loan_balance,total_final_21_limit,total_final_1_limit,total_final_7_limit,previous_21_limit,previous_7_limit,previous_1_limit,model_630_21_limit,model_630_7_limit,model_630_1_limit,repayments_by_dd_vs_principal_mean,due_date_rm_ge_rm_1d,due_date_rm_ge_rm_add_back,repayments_by_erd_vs_principal_mean,rllvr_date_rm_ge_rm_add_back,never_borrowed_flag,adjusted_loan_count,limit_factor_21,limit_factor_7,limit_factor_1,idm_factor_21,idm_factor_7,idm_factor_1,trading_consistency_bands,loan_count_bands,new_limit_factor_21,new_limit_factor_7,new_limit_factor_1,weight_good_loans_repayment_ratio,weight_consistency,weight_recency,risk_rules_factor,ultimate_factor_21,ultimate_factor_7,ultimate_factor_1,limit_21_day,limit_7_day,limit_1_day,adjusted_21_limit,adjusted_7_limit,adjusted_1_limit,final_7_limit,final_1_limit,blacklist_flag,21_day_limit,bf_final_21_limit,af_final_21_limit,af95_final_21_limit,final_21_limit,new_final_21_limit,updated_final_7_limit,updated_final_1_limit,old_final_21_limit,old_final_7_limit,old_final_1_limit,new_final_21_limit_1,new_final_7_limit_1,new_final_1_limit_1,zeroized_7,zeroized_1,updated_7_limit,updated_1_limit,updated_new_7_limit,updated_new_1_limit,zeroize_final_1_limit,update_flag,reinstatement_reason,model_630_max_global_limit,hurdle_rate_dd,hurdle_rate,rllvr_date_rm_ge_rm_never_borrowed,consolidated_repayments_by_dd,consolidated_repayments_by_erd,max_global_limit,previous_max_global_limit,max_limit_diff,model_version,created_at
40648,254792553338,7569213,2022-11-15,2022-12-14,21,16139.37,30,0.70,No_rules_relaxed,70.00,No,0.00,34100807,True,Approve,0.00,254792553338,1,300.00,7.00,1100.00,0.00,2022-12-12,2022-12-19,NaT,2022-12-19,2022-12-22,2023-03-22,65.00,2.00,active_default,442.00,1100.00,2022-12-12,0.00,0.00,72.00,0,65.00,1.00,1.00,7.00,0.00,0.00,1100.00,1100.00,0.00,1100.00,62.00,False,1134.32,1134.32,1134.32,0,2900.00,8400.00,0,1200,0,NaN,NaN,NaN,0.00,0,0,1.00,1,0,1,0.50,0.17,0.17,1.00,1.00,1.00,Band 5,Band 2,0.15,0.12,0.12,0.00,0.00,0,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0.00,0,0,1,3300,0,0.00,0.00,0,0.00,0,0,0,0,0,0.00,0,0,0,0,0,0,0,0,0,0.00,None,NaN,NaN,NaN,1.00,0.00,1.00,0,1200,-1200,"2023-008[2023-01-30, 2023-02-23]",2023-02-23 12:58:49
3520,254723449514,984720,2022-12-13,2022-12-13,1,3.33,1,1.00,No_rules_relaxed,71.00,No,0.00,23405542,True,Reject,0.00,nan,2,300.00,30.00,5000.00,0.00,2019-10-19,2019-11-18,NaT,2019-11-18,NaT,NaT,1192.00,1.00,active_default,NaN,5000.00,2019-10-19,1.00,0.50,1222.00,0,1192.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0.00,0.00,NaN,False,7967.00,0.00,0.00,0,0.00,0.00,0,0,0,NaN,NaN,NaN,NaN,0,0,NaN,0,1,0,0.35,0.12,0.12,0.70,0.71,0.71,Band 7,Band 2,0.25,0.15,0.15,0.00,1.00,0,0.25,0.04,0.03,0.03,0.15,0.09,0.09,0.00,0.00,0.00,0,0,1,0,0,0.00,0.00,0,0.00,0,0,0,0,0,0.00,0,0,0,0,0,0,0,0,0,0.00,None,NaN,0.49,0.58,0.00,0.49,0.58,0,0,0,"2023-008[2023-01-30, 2023-02-23]",2023-02-23 12:58:49


In [7]:
# Data set dimension
df_scoring_stabilisation.shape

(43453, 130)

#### First Time Defaults

In [8]:
%%time
# Data loading
df_ftd_clean = pd.read_parquet(project_dir + ftd_clean_data_path)
df_ftd_clean.sample(2)

CPU times: total: 0 ns
Wall time: 4.09 ms


,store_number,update_flag,reinstatement_reason
36,7399023,1,first time defaulters during election risk mitigation measures
11,7059116,1,first time defaulters during election risk mitigation measures


In [9]:
# Data set dimension
df_ftd_clean.shape

(39, 3)

In [10]:
# Check unique values
df_ftd_clean.store_number.nunique()

39

#### Merge Datasets

In [11]:
# Columns in-scope
cols_in_scope = ['store_number', 'final_21_limit', 'final_7_limit', 'final_1_limit', 
                 "model_630_21_limit", "model_630_7_limit", "model_630_1_limit", "model_630_max_global_limit", 
                 "loan_balance", "blacklist_flag"]

In [12]:
# Merge datasets
ftd_monitoring = df_scoring_stabilisation.merge(df_ftd_clean['store_number'], how="right", on=["store_number"])
ftd_monitoring.sample(2)

,mobile_number,store_number,most_recent_trx_date_past_30_days,last_trx_date,actual_trx_days,approx_30_days_trx_val,expected_trx_days,page_active_days,inference_col,days_since_last_trx,transacted_last_5_days,weight_till_recency,national_id,is_iprs_validated,idm_recommendation,idm_limit,client_mobile_number,loan_count,loan_status,term_frequency,principal_disbursed,principal_repaid,disbursed_on_date,expected_matured_on_date,closed_on_date,due_date_fixed,end_rollover_date_fixed,expected_dpd90,days_past_due,bloom_version,loan_repayment_status,src_crdt_score,max_principal_amount,max_loan_disbursement_date,count_good_loans,good_loans_repayment_ratio,num_days_since_last_disbursement,weight_dpd,max_days_past_due,loan_count_past_3_months,count_7_day_loans,avg_loan_tenure,count_7_day_loans_paid_upto_rollover,good_loans_repayment_ratio(7_day_loans),minimum_7_day_principal_disbursed,total_sum_7_day_principal_disbursed,total_sum_1_day_principal_disbursed,avg_7_day_principal_disbursed,days_past_end_rollover,any_bloom2_1day,total_outstanding_sum,safaricom_loan_balance_sum,loan_balance,total_final_21_limit,total_final_1_limit,total_final_7_limit,previous_21_limit,previous_7_limit,previous_1_limit,model_630_21_limit,model_630_7_limit,model_630_1_limit,repayments_by_dd_vs_principal_mean,due_date_rm_ge_rm_1d,due_date_rm_ge_rm_add_back,repayments_by_erd_vs_principal_mean,rllvr_date_rm_ge_rm_add_back,never_borrowed_flag,adjusted_loan_count,limit_factor_21,limit_factor_7,limit_factor_1,idm_factor_21,idm_factor_7,idm_factor_1,trading_consistency_bands,loan_count_bands,new_limit_factor_21,new_limit_factor_7,new_limit_factor_1,weight_good_loans_repayment_ratio,weight_consistency,weight_recency,risk_rules_factor,ultimate_factor_21,ultimate_factor_7,ultimate_factor_1,limit_21_day,limit_7_day,limit_1_day,adjusted_21_limit,adjusted_7_limit,adjusted_1_limit,final_7_limit,final_1_limit,blacklist_flag,21_day_limit,bf_final_21_limit,af_final_21_limit,af95_final_21_limit,final_21_limit,new_final_21_limit,updated_final_7_limit,updated_final_1_limit,old_final_21_limit,old_final_7_limit,old_final_1_limit,new_final_21_limit_1,new_final_7_limit_1,new_final_1_limit_1,zeroized_7,zeroized_1,updated_7_limit,updated_1_limit,updated_new_7_limit,updated_new_1_limit,zeroize_final_1_limit,update_flag,reinstatement_reason,model_630_max_global_limit,hurdle_rate_dd,hurdle_rate,rllvr_date_rm_ge_rm_never_borrowed,consolidated_repayments_by_dd,consolidated_repayments_by_erd,max_global_limit,previous_max_global_limit,max_limit_diff,model_version,created_at
23,254740400224,7449540,2022-11-15,2022-12-14,29,93366.75,30,0.97,No_rules_relaxed,70.00,No,0.00,29305144,True,Approve,0.00,254740400224,8,300.00,7.00,19400.00,0.00,2022-07-30,2022-08-06,NaT,2022-08-06,2022-08-09,2022-11-07,200.00,2.00,active_default,480.00,19400.00,2022-07-30,7.00,0.88,207.00,0,200.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0.00,0.00,197.00,False,30396.43,39269.98,39269.98,162700,131400.00,135000.00,32600,18400,18400,32600.00,18400.00,18400.00,0.77,0,0,0.00,0,0,0,0.50,0.17,0.17,1.00,1.00,1.00,Band 7,Band 5,0.40,0.23,0.23,0.00,1.00,0,0.25,0.10,0.06,0.06,9336.68,5251.88,5251.88,0.00,0.00,0.00,0,0,1,0,0,0.00,0.00,0,0.00,0,0,0,0,0,0.00,0,0,0,0,0,0,0,0,0,1.00,first time defaulters during election risk mitigation measures,32600.00,NaN,NaN,1.00,0.77,0.00,0,32600,-32600,"2023-008[2023-01-30, 2023-02-23]",2023-02-23 12:58:49
12,254742373427,7653923,2022-11-15,2022-12-14,29,23664.26,30,0.97,No_rules_relaxed,70.00,No,0.00,38029883,True,Reject,0.00,254742373427,6,300.00,7.00,5000.00,0.00,2022-07-29,2022-08-05,NaT,2022-08-05,2022-08-08,2022-11-06,201.00,2.00,active_default,423.00,5000.00,2022-07-29,5.00,0.83,208.00,0,201.00,0.00,0.00,NaN,0.00,0.00,0.00,0.00,0.00,0.00,198.00,False,7826.63,13044.87,13044.87,38800,27400.00,27400.00,9700,5600,5600,9700.00,5600.00,5600.00,0.68,0,0,0.00,0,0,0,0.35,0.12,0.12,0.70,0.71,0.71,Band 7,Band 4,0.35,0.20,0.20,0.00,1.00,0,0.25,0.06,0.04,0.04,1449.44,835.21,835.21,0.00,0.00,0.00,0,0,1,0,0,0.00,0.0

In [13]:
# Data set dimension
ftd_monitoring.shape

(39, 130)

#### Diagnostics

In [14]:
# With limits
wl = ftd_monitoring.loc[(ftd_monitoring['blacklist_flag'] == 0)]
print(wl.shape)
wl[cols_in_scope].sample(2)

(26, 130)


,store_number,final_21_limit,final_7_limit,final_1_limit,model_630_21_limit,model_630_7_limit,model_630_1_limit,model_630_max_global_limit,loan_balance,blacklist_flag
22,7153772,6300,3600,3600,6300.00,3600.00,3600.00,6300.00,5382.00,0
31,7522329,0,11000,11000,0.00,11000.00,11000.00,11000.00,7943.25,0


In [15]:
# Without limits not in model_630
wol_ms = ftd_monitoring.loc[(ftd_monitoring['blacklist_flag'] == 1) & (ftd_monitoring['model_630_21_limit'].isna())]
print(wol_ms.shape)
wol_ms[cols_in_scope].head(2)

(0, 130)


,store_number,final_21_limit,final_7_limit,final_1_limit,model_630_21_limit,model_630_7_limit,model_630_1_limit,model_630_max_global_limit,loan_balance,blacklist_flag


In [16]:
# Without limits balance greater than limit
wol_l = ftd_monitoring.loc[(ftd_monitoring['blacklist_flag'] == 1) & ~(ftd_monitoring['model_630_21_limit'].isna())]
print(wol_l.shape)
wol_l[cols_in_scope].sample(2)

(13, 130)


,store_number,final_21_limit,final_7_limit,final_1_limit,model_630_21_limit,model_630_7_limit,model_630_1_limit,model_630_max_global_limit,loan_balance,blacklist_flag
35,7156825,0,0,0,28600.00,16200.00,16200.00,28600.00,46048.07,1
13,632642,0,0,0,31000.00,17900.00,17900.00,31000.00,36605.21,1


In [17]:
# Without limits
inv = ftd_monitoring.loc[(ftd_monitoring['blacklist_flag'].isna())]
print(inv.shape)
inv[cols_in_scope].head()

(0, 130)


,store_number,final_21_limit,final_7_limit,final_1_limit,model_630_21_limit,model_630_7_limit,model_630_1_limit,model_630_max_global_limit,loan_balance,blacklist_flag


In [18]:
# Data set dimension
wl.shape[0] + wol_ms.shape[0] + wol_l.shape[0] + inv.shape[0]

39